## parallel_in_python3.ipynb

### time cells

In [5]:
import sys
import os
import os.path as op
from glob import glob
import shutil
from collections import OrderedDict as od
from importlib import reload
import numpy as np
import pandas as pd
import statsmodels.api as sm
from time import time
from cluster_helper.cluster import cluster_view
sys.path.append('/home1/dscho/code/general')
import data_io as dio 
from time_cells import spike_sorting, spike_preproc, events_preproc, events_proc, time_bin_analysis, remapping, pop_decoding, time_cell_plots

In [6]:
def classify_time_bins_parallel(ii):
    import sys
    import os
    from time import sleep
    import numpy as np
    sys.path.append('/home1/dscho/code/general')
    import data_io as dio 
    sys.path.append('/home1/dscho/code/projects/time_cells')
    import time_bin_analysis
    
    # Take a nap.
    sleep_secs = int(60 * np.random.rand())
    sleep(sleep_secs)
    
    try:
        n_perms = 10
        spikes, event_times = dio.open_pickle('/home1/dscho/projects/time_cells/analysis/classifiers/tmp_inputs.pkl')
        
        analysis_dir = '/home1/dscho/projects/time_cells/analysis'
        output_f = os.path.join(analysis_dir, 'classifiers', 'LinearSVM-Delay1_Delay2-20_time_bins-{}perms-{}.pkl'.format(n_perms, ii))
        output = time_bin_analysis.classify_time_bins(spikes, 
                                                      event_times, 
                                                      ['Delay1', 'Delay2'], 
                                                      n_time_bins=20, 
                                                      k=6, 
                                                      n_perms=n_perms,
                                                      save_as=output_f)
    except:
        err = sys.exc_info()
        errf = '/home1/dscho/logs/TryExceptError-classify_time_bins_parallel_{}'.format(ii)
        os.system('touch {}'.format(errf))
        with open(errf, 'w') as f:
            f.write(str(err) + '\n')
            
    return None


def fr_by_time_vs_null_parallel(info):
    import sys
    import os
    import warnings
    warnings.filterwarnings('ignore')
    sys.path.append('/home1/dscho/code/general')
    sys.path.append('/home1/dscho/code/projects/time_cells')
    import data_io as dio
    import events_preproc
    import time_bin_analysis
    from time import sleep
    import numpy as np
    
    # Take a nap.
    sleep_secs = int(300 * np.random.rand())
    sleep(sleep_secs)
    
    try:
        subj_sess = info['subj_sess']
        chan = info['chan']
        unit = info['unit']
        game_states = info['game_states']
        n_time_bins = info['n_time_bins']

        n_perms = 1000
        analysis_dir = '/scratch/dscho/time_cells/analysis/'
        input_f = os.path.join(analysis_dir, 'spikes', 
                               '{}-CSC{}-unit{}-spikes.pkl'
                               .format(subj_sess, chan, unit))
        fr_train = dio.open_pickle(input_f)['fr_train']
        event_times = events_preproc.create_event_time_bins(subj_sess)
        output_f = os.path.join(analysis_dir, 'spikes_by_time_bin',
                                '{}-CSC{}-unit{}-{}-spikes_by_time_bin.pkl'
                                .format(subj_sess, chan, unit, '_'.join(game_states)))

        output = time_bin_analysis.fr_by_time_vs_null(fr_train=fr_train, 
                                                      event_times=event_times, 
                                                      game_states=game_states, 
                                                      n_time_bins=n_time_bins, 
                                                      n_perms=n_perms,
                                                      save_as=output_f)
    except:
        err = sys.exc_info()
        errf = '/home1/dscho/logs/TryExceptError-save_phase_vectors_parallel-{}-chan{}-unit{}-{}'.format(subj_sess, chan, unit, '_'.join(game_states))
        os.system('touch {}'.format(errf))
        with open(errf, 'w') as f:
            f.write(str(err) + '\n')
            
    return None


def event_fr_parallel(spikes_f):
    import sys
    import os
    from time import sleep
    import numpy as np
    sys.path.append('/home1/dscho/code/general')
    import data_io as dio 
    sys.path.append('/home1/dscho/code/projects/time_cells')
    import events_preproc
    import spike_preproc
    
    # Take a nap.
    sleep_secs = int(360 * np.random.rand())
    sleep(sleep_secs)
    
    try:
        # Load spike data for the neuron.
        spikes = dio.open_pickle(spikes_f)
        subj_sess = spikes['subj_sess']

        # Load events and event times.
        events = events_preproc.format_events(subj_sess,
                                              overwrite=False,
                                              save_output=False,
                                              verbose=False)
#         trial_times = events_preproc.trial_intervals(events)
        event_times = events_preproc.create_event_time_bins(subj_sess,
                                                            events=events,
                                                            overwrite=True,
                                                            save_output=False,
                                                            verbose=False)

        # Calculate the firing rate for each trial and trial phase time bin.
        output = spike_preproc.event_fr(spikes,
#                                         trial_times=trial_times, 
                                        event_times=event_times,
#                                         include_trial_null=True,
                                        include_event_null=True,
                                        sigma=500,
                                        overwrite=False,
                                        save_output=True,
                                        verbose=False)
        
    except:
        err = sys.exc_info()
        errf = '/home1/dscho/logs/TryExceptError-event_fr_parallel-{}'.format('-'.join(os.path.basename(spikes_f[0]).split('-')[:3]))
        os.system('touch {}'.format(errf))
        with open(errf, 'w') as f:
            f.write(str(err) + '\n')
    
    return None


def calc_mean_fr_by_time_parallel(f):
    import sys
    import os
    sys.path.append('/home1/dscho/code/projects/time_cells')
    import time_bin_analysis
    
    try:
        output = time_bin_analysis.calc_mean_fr_by_time(f,
                                                        overwrite=False,
                                                        save_output=True,
                                                        verbose=False)
    except:
        err = sys.exc_info()
        errf = '/home1/dscho/logs/TryExceptError-calc_mean_fr_by_time_parallel-{}'.format('-'.join(os.path.basename(f[0]).split('-')[:3]))
        os.system('touch {}'.format(errf))
        with open(errf, 'w') as f:
            f.write(str(err) + '\n')
    
    return None

def calc_fr_by_time_and_pos_parallel(f):
    import sys
    import os
    import inspect
    import traceback
    sys.path.append('/home1/dscho/code/general')
    sys.path.append('/home1/dscho/code/projects')
    import data_io as dio
    from time_cells.place_analysis import calc_fr_by_time_and_pos
    
    try:
        output = calc_fr_by_time_and_pos(dio.open_pickle(f),
                                         compute_null=True,
                                         compute_stats=True,
                                         overwrite=True,
                                         save_output=True,
                                         verbose=False)
    except:
        func_name = inspect.stack()[1][3]
        neuron_id = '-'.join(os.path.basename(f).split('-')[:3])
        errf = os.path.join('/home1/dscho/logs', '{}-{}'.format(func_name, neuron_id))
        os.system('touch {}'.format(errf))
        with open(errf, 'w') as f:
            err = traceback.format_exc()
            f.write(err + '\n')
    
    return None

def compare_fr_by_time_and_pos_parallel(f):
    import sys
    import os
    sys.path.append('/home1/dscho/code/general')
    import data_io as dio
    sys.path.append('/home1/dscho/code/projects')
    from time_cells.place_analysis import compare_fr_by_time_and_pos
    
    try:
        output = compare_fr_by_time_and_pos(dio.open_pickle(f),
                                            overwrite=False,
                                            save_output=True,
                                            verbose=False)
    except:
        err = sys.exc_info()
        neuron_id = '-'.join(os.path.basename(f).split('-')[:3])
        errf = '/home1/dscho/logs/TryExceptError-compare_fr_by_time_and_pos_parallel-{}'.format('-'.join(os.path.basename(f).split('-')[:3]))
        os.system('touch {}'.format(errf))
        with open(errf, 'w') as f:
            f.write(str(err) + '\n')
    
    return None

def glm_fit_unit_parallel(subj_sess_neuron):
    import sys
    import os
    from time import sleep
    import numpy as np
    sys.path.append('/home1/dscho/code/projects')
    from time_cells.time_bin_analysis import glm_fit_unit
    
    # Take a nap.
    if False:
        sleep_secs = int(600 * np.random.rand())
        sleep(sleep_secs)
    
    try:
        subj_sess, chan, unit = subj_sess_neuron.split('-')
        neuron = '{}-{}'.format(chan, unit)
        output = glm_fit_unit(subj_sess,
                              neuron,
                              n_perm=1000,
                              overwrite=False,
                              save_output=True)
    except:
        err = sys.exc_info()
        errf = '/home1/dscho/logs/TryExceptError-glm_fit_unit_parallel-{}'.format(subj_sess_neuron)
        os.system('touch {}'.format(errf))
        with open(errf, 'w') as f:
            f.write(str(err) + '\n')
    
    return None

def model_unit_fr_parallel(subj_sess_neuron):
    import sys
    import os
    from time import sleep
    import numpy as np
    sys.path.append('/home1/dscho/code/projects')
    from time_cells.time_bin_analysis import model_unit_fr
    
    # Take a nap.
    if True:
        sleep_secs = int(1800 * np.random.rand())
        sleep(sleep_secs)
    
    try:
        subj_sess, chan, unit = subj_sess_neuron.split('-')
        neuron = '{}-{}'.format(chan, unit)
        output = model_unit_fr(subj_sess,
                               neuron,
                               model='ols',
                               n_perm=1000,
                               overwrite=False,
                               save_output=True)
    except:
        err = sys.exc_info()
        errf = '/home1/dscho/logs/TryExceptError-model_unit_fr_parallel-{}'.format(subj_sess_neuron)
        os.system('touch {}'.format(errf))
        with open(errf, 'w') as f:
            f.write(str(err) + '\n')
    
    return None

def game_state_unit_fr_diff_parallel(subj_sess_neuron):
    import sys
    import os
    from time import sleep
    import numpy as np
    sys.path.append('/home1/dscho/code/projects')
    from time_cells.trial_phase_analysis import game_state_unit_fr_diff
    
    # Take a nap.
    if False:
        sleep_secs = int(300 * np.random.rand())
        sleep(sleep_secs)
    
    try:
        subj_sess, chan, unit = subj_sess_neuron.split('-')
        neuron = '{}-{}'.format(chan, unit)
        output = game_state_unit_fr_diff(neuron,
                                         subj_sess=subj_sess,
                                         n_perm=1000,
                                         regress_trial=True,
                                         perm_method='circshift',
                                         alpha=0.05,
                                         overwrite=False,
                                         save_output=True)
    except:
        err = sys.exc_info()
        errf = '/home1/dscho/logs/TryExceptError-game_state_unit_fr_diff-{}'.format(subj_sess_neuron)
        os.system('touch {}'.format(errf))
        with open(errf, 'w') as f:
            f.write(str(err) + '\n')
    
    return None

def run_ols_delay_parallel(subj_sess_unit):
    import sys
    import os
    sys.path.append('/home1/dscho/code/projects')
    from time_cells import time_bin_analysis
    
    proj_dir = '/home1/dscho/projects/time_cells'
    n_perm = 10000
    
    try:
        mod_pairs, ols_weights = time_bin_analysis.run_ols_delay(subj_sess_unit,
                                                                 n_perm=n_perm,
                                                                 alpha=0.05,
                                                                 save_output=True,
                                                                 overwrite=False)
    except:
        err = sys.exc_info()
        errf = '/home1/dscho/logs/TryExceptError-run_ols_delay_parallel-{}'.format(subj_sess_unit)
        os.system('touch {}'.format(errf))
        with open(errf, 'w') as f:
            f.write(str(err) + '\n')
            
def run_ols_nav_parallel(subj_sess_unit):
    import sys
    import os
    sys.path.append('/home1/dscho/code/projects')
    from time_cells import time_bin_analysis
    
    proj_dir = '/home1/dscho/projects/time_cells'
    n_perm = 10000
    
    try:
        mod_pairs, ols_weights = time_bin_analysis.run_ols_nav(subj_sess_unit,
                                                               n_perm=n_perm,
                                                               alpha=0.05,
                                                               save_output=True,
                                                               overwrite=False)
    except:
        err = sys.exc_info()
        errf = '/home1/dscho/logs/TryExceptError-run_ols_nav_parallel-{}'.format(subj_sess_unit)
        os.system('touch {}'.format(errf))
        with open(errf, 'w') as f:
            f.write(str(err) + '\n')

In [ ]:
# Get neurons to process.
proj_dir = '/home1/dscho/projects/time_cells'
fpath = op.join(proj_dir, 'analysis', 'unit_to_behav', '{}-Encoding_Retrieval-ols_model_pairs.pkl')
pop_spikes = pop_decoding.load_pop_spikes()
neurons = [neuron for neuron in pop_spikes.neurons if not op.exists(fpath.format(neuron))]
print('{} neurons to process'.format(len(neurons)))

# Parallel processing
n_ops = len(neurons)
print('Running code for {} operations.\n'.format(n_ops))
with cluster_view(scheduler="sge", queue="RAM.q", num_jobs=np.min((n_ops, 200)), cores_per_job=1) as view:
    output = view.map(run_ols_nav_parallel, neurons)

457 neurons to process
Running code for 457 operations.

152 Engines running


In [38]:
from time_cells import trial_phase_analysis 
reload(trial_phase_analysis)
from time_cells.trial_phase_analysis import game_state_unit_fr_diff

In [34]:
start_time = time()

subj_sess_neuron = subj_sess_neurons[0]
print(subj_sess_neuron)
_ = game_state_unit_fr_diff_parallel(subj_sess_neuron)
model_pairs = dio.open_pickle(op.join(proj_dir, 'analysis', 'unit_to_behav', '{}-ols-game_state-model_pairs.pkl'.format(subj_sess_neuron)))

print('Done in {:.1f}s'.format(time() - start_time))

In [10]:
# Load event_spikes and get the names of each neuron.
proj_dir = '/home1/dscho/projects/time_cells'

# Get sessions.
sessions = np.unique([op.basename(f).split('-')[0] 
                      for f in glob(op.join(proj_dir, 'analysis', 'events', '*.pkl'))])
print('{} subjects, {} sessions'.format(len(np.unique([x.split('_')[0] for x in sessions])), len(sessions)))

subj_sess_neurons = []
for subj_sess in sessions:
    event_spikes = time_bin_analysis.load_event_spikes(subj_sess, verbose=False)
    for neuron in event_spikes.column_map['neurons']:
        subj_sess_neuron = '{}-{}'.format(subj_sess, neuron)
        output_f = op.join(proj_dir, 'analysis', 'unit_to_behav', '{}-ols-game_state-model_pairs.pkl'.format(subj_sess_neuron))
        if not op.exists(output_f):
            subj_sess_neurons.append(subj_sess_neuron)
print('{} neurons'.format(len(subj_sess_neurons)))

# Parallel processing
n_ops = len(subj_sess_neurons)
print('Running code for {} operations.\n'.format(n_ops))
with cluster_view(scheduler="sge", queue="RAM.q", num_jobs=np.min((n_ops, 200)), cores_per_job=1) as view:
    output = view.map(game_state_unit_fr_diff_parallel, subj_sess_neurons)

8 subjects, 10 sessions
Loading saved Events file
13 neurons
Running code for 13 operations.

13 Engines running
Sending a shutdown signal to the controller and engines.


In [7]:
def model_unit_fr_parallel(subj_sess_neuron):
    import sys
    import os
    from time import sleep
    import numpy as np
    sys.path.append('/home1/dscho/code/projects')
    from time_cells.time_bin_analysis import model_unit_fr
    
    # Take a nap.
    if False:
        sleep_secs = int(1800 * np.random.rand())
        sleep(sleep_secs)
    try:
        subj_sess, chan, unit = subj_sess_neuron.split('-')
        neuron = '{}-{}'.format(chan, unit)
        output = model_unit_fr(subj_sess,
                               neuron,
                               model='ols',
                               n_perm=1000,
                               overwrite=False,
                               save_output=True)
    except:
        err = sys.exc_info()
        errf = '/home1/dscho/logs/TryExceptError-model_unit_fr_parallel-{}'.format(subj_sess_neuron)
        os.system('touch {}'.format(errf))
        with open(errf, 'w') as f:
            f.write(str(err) + '\n')
    
    return None

start_time = time()

# Load event_spikes and get the names of each neuron.
proj_dir = '/home1/dscho/projects/time_cells'

# Get sessions.
sessions = np.unique([op.basename(f).split('-')[0] 
                      for f in glob(op.join(proj_dir, 'analysis', 'events', '*.pkl'))])
print('{} subjects, {} sessions'.format(len(np.unique([x.split('_')[0] for x in sessions])), len(sessions)))

subj_sess_neurons = []
for subj_sess in sessions:
    event_spikes = time_bin_analysis.load_event_spikes(subj_sess, verbose=False)
    for neuron in event_spikes.column_map['neurons']:
        subj_sess_neuron = '{}-{}'.format(subj_sess, neuron)
        #output_f = op.join(proj_dir, 'analysis', 'behav_glms', '{}-glm_results.pkl'.format(subj_sess_neuron))
        output_f = op.join(proj_dir, 'analysis', 'unit_to_behav', '{}-ols-time_bin-model_pairs.pkl'.format(subj_sess_neuron))
        if not op.exists(output_f):
            subj_sess_neurons.append(subj_sess_neuron)
print('{} neurons'.format(len(subj_sess_neurons)))

# Parallel processing
for subj_sess_neuron in subj_sess_neurons:
    _start_time = time()
    print(subj_sess_neuron)
    _ = model_unit_fr_parallel(subj_sess_neuron)
    print('{:.1f}s'.format(time() - start_time))

print('Done in {:.1f}s'.format(time() - start_time))

10 subjects, 12 sessions
2 neurons
U540_ses0-5-1
Loading saved EventSpikes file
494.8s
U540_ses0-34-1
Loading saved EventSpikes file
972.9s
Done in {:.1f}s


In [4]:
# Load event_spikes and get the names of each neuron.
proj_dir = '/home1/dscho/projects/time_cells'

# Get sessions.
sessions = np.unique([op.basename(f).split('-')[0] 
                      for f in glob(op.join(proj_dir, 'analysis', 'events', '*.pkl'))])
print('{} subjects, {} sessions'.format(len(np.unique([x.split('_')[0] for x in sessions])), len(sessions)))

subj_sess_neurons = []
for subj_sess in sessions:
    event_spikes = time_bin_analysis.load_event_spikes(subj_sess, verbose=False)
    for neuron in event_spikes.column_map['neurons']:
        subj_sess_neuron = '{}-{}'.format(subj_sess, neuron)
        #output_f = op.join(proj_dir, 'analysis', 'behav_glms', '{}-glm_results.pkl'.format(subj_sess_neuron))
        output_f = op.join(proj_dir, 'analysis', 'unit_to_behav', '{}-ols-time_bin-model_pairs.pkl'.format(subj_sess_neuron))
        if not op.exists(output_f):
            subj_sess_neurons.append(subj_sess_neuron)
print('{} neurons'.format(len(subj_sess_neurons)))

# Parallel processing
n_ops = len(subj_sess_neurons)
print('Running code for {} operations.\n'.format(n_ops))
with cluster_view(scheduler="sge", queue="RAM.q", num_jobs=np.min((n_ops, 100)), cores_per_job=2) as view:
    output = view.map(model_unit_fr_parallel, subj_sess_neurons)

10 subjects, 12 sessions
43 neurons
Running code for 43 operations.

4 Engines running
Sending a shutdown signal to the controller and engines.


OSError: [Errno 16] Device or resource busy: '.nfs0000004402d418120000aa23'

In [54]:
# files = glob('/home1/dscho/projects/time_cells/analysis/fr_by_time_bin/mean_fr*.pkl')
# for f in files:
#     d, fname = os.path.split(f)
#     s, c, u = fname.replace('mean_fr_by_time_', '').replace('ch', 'CSC').replace('.pkl', '').split('-')
#     fnew = os.path.join(d, '{}-{}-unit{}-mean_fr_by_time.pkl'.format(s, c, u))
#     shutil.move(f, fnew)

In [ ]:
# subj_sess = 'U530_ses0'
# proj_dir = '/home1/dscho/projects/time_cells'

# files = glob(os.path.join(proj_dir, 'analysis', 'spikes', '{}*unit*.pkl'.format(subj_sess)))
# print('{} spike files'.format(len(files)))

# # Parallel processing
# n_ops = len(files)
# print('Running code for {} operations.\n'.format(n_ops))
# with cluster_view(scheduler="sge", queue="RAM.q", num_jobs=n_ops, cores_per_job=1) as view:
#     output = view.map(event_fr_parallel, files)

67 spike files
Running code for 67 operations.

33 Engines running


In [14]:
proj_dir = '/home1/dscho/projects/time_cells'

files = np.sort(glob(os.path.join(proj_dir, 'analysis', 'spikes', 'U518_ses1*unit*.pkl')))
print('{} files'.format(len(files)))

# Parallel processing
n_ops = len(files)
print('Running code for {} operations.\n'.format(n_ops))
with cluster_view(scheduler="sge", queue="RAM.q", num_jobs=np.min((len(files), 150)), cores_per_job=1) as view:
    output = view.map(calc_fr_by_time_and_pos_parallel, files)

43 files


In [25]:
# proj_dir = '/home1/dscho/projects/time_cells'

# files = np.sort(glob(os.path.join(proj_dir, 'analysis', 'fr_by_time_and_pos', 'U530_ses1*fr_by_time_and_pos.pkl')))
# print('{} files'.format(len(files)))

# # Parallel processing
# n_ops = len(files)
# print('Running code for {} operations.\n'.format(n_ops))
# with cluster_view(scheduler="sge", queue="RAM.q", num_jobs=n_ops, cores_per_job=1) as view: 
#     output = view.map(compare_fr_by_time_and_pos_parallel, files)

In [12]:
# # Z-score trial phase time bins and calculate temporal info scores.
# start_time = time()

# game_states = [['Prepare1'], ['Delay1'], ['Encoding'], ['Prepare2'], ['Delay2'], ['Retrieval'],
#                ['Prepare1', 'Prepare2'], ['Delay1', 'Delay2'], ['Encoding', 'Retrieval']]
# n_perm = 1000
# verbose = True

# # Load FR files.
# proj_dir = '/home1/dscho/projects/time_cells'
# files = np.sort(glob(os.path.join(proj_dir, 'analysis', 'fr_by_time_bin', '*unit*.pkl')))
# print('{} files'.format(len(files)))

# mean_fr_by_time = []
# cols = ['subj_sess', 'subj', 'sess', 'chan', 'unit', 'gameState', 
#         'fr', 'z_fr', 'z_fr_max', 'z_fr_max_ind', 'tis', 'z_tis', 'pval']
# for i, f in enumerate(files):
#     if np.all((verbose, i > 0, i % 20 == 0)):
#         print('{}/{}...'.format(i, len(files)))
#     neuron = '-'.join(os.path.basename(f).split('-')[:3]).replace('CSC', 'ch').replace('unit', '')
#     subj_sess, chan, unit = neuron.split('-')
#     chan = chan[2:]
#     subj, sess = subj_sess.split('_')
#     event_times_ = dio.open_pickle(f)['event_times']
#     for game_state in game_states:
#         if not np.all(np.isin(game_state, event_times_['gameState'].unique())):
#             continue
#         obs = np.mean(event_times_
#                       .query("(gameState=={})"
#                       .format(game_state))['fr']
#                       .tolist(), axis=0)
#         null_mean = np.mean(np.mean(event_times_
#                                     .query("(gameState=={})"
#                                     .format(game_state))['fr_null']
#                                     .tolist(), axis=0), axis=0)
#         null_std = np.std(np.mean(event_times_
#                                   .query("(gameState=={})"
#                                   .format(game_state))['fr_null']
#                                   .tolist(), axis=0), axis=0)
#         obs_z = (obs - null_mean) / null_std
        
#         tis = time_bin_analysis.info_rate(obs)  # temporal information score
#         null_tis = np.array([time_bin_analysis.info_rate(np.mean(event_times_
#                                                                  .query("(gameState=={})"
#                                                                  .format(game_state))['fr_null']
#                                                                  .tolist(), axis=0)[iPerm, :])
#                              for iPerm in range(n_perm)])
#         tis_z = (tis - np.mean(null_tis)) / np.std(null_tis)
#         pval = (np.sum(null_tis >= tis) + 1) / (n_perm + 1)
        
#         mean_fr_by_time.append([subj_sess, subj, sess, chan, unit, '_'.join(game_state), 
#                                 obs, obs_z, obs_z.max(), obs_z.argmax(), tis, tis_z, pval])

# mean_fr_by_time = pd.DataFrame(mean_fr_by_time, columns=cols)

# # Save the output.
# output_f = os.path.join(proj_dir, 'analysis', 'fr_by_time_bin', 
#                         'mean_fr_by_time_{}_neurons.pkl'.format(len(files)))
# dio.save_pickle(mean_fr_by_time, output_f)

# print('Done in {:.2f}s'.format(time() - start_time))

135 files


/home1/dscho/code/projects/time_cells/time_bin_analysis.py:277: RuntimeWarning: divide by zero encountered in log2
  bits_per_spike = np.nansum(prob_x * (fr_given_x/mean_fr) * np.log2(fr_given_x/mean_fr))
/home1/dscho/code/projects/time_cells/time_bin_analysis.py:277: RuntimeWarning: invalid value encountered in multiply
  bits_per_spike = np.nansum(prob_x * (fr_given_x/mean_fr) * np.log2(fr_given_x/mean_fr))


20/135...
40/135...
60/135...
80/135...
100/135...
120/135...
Saved /home1/dscho/projects/time_cells/analysis/fr_by_time_bin/mean_fr_by_time_135_neurons.pkl
Done in 8964.66s


In [23]:
def format_info(x, game_states):
    x = os.path.basename(x).split('-')
    subj_sess = x[0]
    chan = x[1][3:]
    unit = x[2][4:]
    if ('Delay1' in game_states) or ('Delay1' in game_states):
        n_time_bins = 20
    elif ('Encoding' in game_states) or ('Retrieval' in game_states):
        n_time_bins = 60
    output = od([('subj_sess', subj_sess),
                 ('chan', chan),
                 ('unit', unit),
                 ('game_states', game_states),
                 ('n_time_bins', n_time_bins)])
    return output

analysis_dir = '/scratch/dscho/time_cells/analysis'
spike_files = glob(os.path.join(analysis_dir, 'spikes', '*CSC*.pkl'))

neurons_to_process = ([format_info(x, ['Delay1', 'Delay2']) for x in spike_files] + 
                      [format_info(x, ['Encoding', 'Retrieval']) for x in spike_files])
print(len(neurons_to_process))

244


In [ ]:
# Parallel processing
print('Running code for {} operations.\n'.format(len(neurons_to_process)))
with cluster_view(scheduler="sge", queue="RAM.q", num_jobs=100, cores_per_job=1) as view: 
    output = view.map(fr_by_time_vs_null_parallel, neurons_to_process)

Running code for 244 operations.

100 Engines running
Sending a shutdown signal to the controller and engines.


In [51]:
# Get event times.
analysis_dir = '/home1/dscho/projects/time_cells/analysis'
event_files = glob(os.path.join(analysis_dir, 'events', '*.pkl'))
sessions =  [os.path.basename(f).split('-')[0] for f in event_files]
event_times = pd.concat([events_preproc.create_event_time_bins(subj_sess) for subj_sess in sessions], axis=0)

# Gather spike info for each neuron.
unit_df = spike_preproc.get_unit_df()
cols = ['subj_sess', 'chan', 'unit', 'n_spikes', 'fr', 'hemroi', 'fr_train']
spikes = pd.DataFrame([dio.open_pickle(row['spikes_f'])[cols] for idx, row in unit_df.iterrows()]).reset_index(drop=True)

# Add time cell info for each neuron, separately for delay and task periods.
spikes_by_time_delay = pd.DataFrame([dio.open_pickle(row['fr_by_time_delay_f']) for idx, row in unit_df.iterrows()])
spikes_by_time_nav = pd.DataFrame([dio.open_pickle(row['fr_by_time_nav_f']) for idx, row in unit_df.iterrows()])
spikes_by_time_delay.columns = ['{}_delay'.format(c) for c in spikes_by_time_delay.columns]
spikes_by_time_nav.columns = ['{}_nav'.format(c) for c in spikes_by_time_nav.columns]
spikes = pd.concat((spikes, spikes_by_time_delay, spikes_by_time_nav), axis=1)

# FDR correct p-values.
alpha = 0.05
spikes['temporal_info_sig_delay'] = sm.stats.multipletests(spikes['temporal_info_pval_delay'], alpha=alpha, method='fdr_tsbky', is_sorted=False, returnsorted=False)[0]
spikes['temporal_info_sig_nav'] = sm.stats.multipletests(spikes['temporal_info_pval_nav'], alpha=alpha, method='fdr_tsbky', is_sorted=False, returnsorted=False)[0]

# Save these objects.
dio.save_pickle([spikes.query("(temporal_info_sig_delay==True)"), event_times], '/home1/dscho/projects/time_cells/analysis/classifiers/tmp_inputs.pkl')

In [68]:
# Parallel processing
print('Running code for {} operations.\n'.format(100))
with cluster_view(scheduler="sge", queue="RAM.q", num_jobs=100, cores_per_job=1) as view: 
    output = view.map(classify_time_bins_parallel, np.arange(100))

Running code for 100 operations.

60 Engines running
Sending a shutdown signal to the controller and engines.


OSError: [Errno 16] Device or resource busy: '.nfs0000004500031d82000006a4'